In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


**Upload it on your Google Drive, read it from within CoLab (10 Marks)**

In [ ]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/DeepLearningbySirSher/newdatasets/bbcsports.csv")
print(df.head())


   Unnamed: 0                                               text   label
0           0  Sharapova overcomes tough Molik\n\nWimbledon c...  tennis
1           1  GB players warned over security\n\nBritain's D...  tennis
2           2  Federer wins title in Rotterdam\n\nWorld numbe...  tennis
3           3  Mauresmo fights back to win title\n\nWorld num...  tennis
4           4  Agassi into second round in Dubai\n\nFourth se...  tennis


In [7]:
print(df['label'].value_counts())

label
football     265
rugby        147
cricket      124
athletics    101
tennis       100
Name: count, dtype: int64


**prepare data that you can feed to dense neutral network, remember text cannot directly be injected to the network, you need to convert it in 0-1 values. Use tokenizer and other classes to prepare the data. (5 Marks)**

In [10]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['label'])
y = to_categorical(y)

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['text'])


sequences = tokenizer.texts_to_sequences(df['text'])
X = pad_sequences(sequences, maxlen=200)

**Use 80% of each of the class labels to train a dense network with appropriate input and output layers and two hidden layers with 64 and 32 neuron respectively (5 Marks)**

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print("Training shape:", X_train.shape)
print("Testing shape:", X_test.shape)

Training shape: (589, 200)
Testing shape: (148, 200)


In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

model = Sequential([
    Embedding(input_dim=10000, output_dim=64),
    GlobalAveragePooling1D(),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_1      │ ?                      │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=10, batch_size=32)


Epoch 1/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.3416 - loss: 1.5939 - val_accuracy: 0.3581 - val_loss: 1.5352
Epoch 2/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.3834 - loss: 1.4903 - val_accuracy: 0.3581 - val_loss: 1.4378
Epoch 3/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.3766 - loss: 1.3700 - val_accuracy: 0.3649 - val_loss: 1.3220
Epoch 4/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.4686 - loss: 1.1774 - val_accuracy: 0.5270 - val_loss: 1.1136
Epoch 5/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6991 - loss: 0.9247 - val_accuracy: 0.7297 - val_loss: 0.8429
Epoch 6/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8142 - loss: 0.6384 - val_accuracy: 0.7973 - val_loss: 0.6090
Epoch 7/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8639 - loss: 0.4140 - val_accuracy: 0.8986 - val_loss: 0.4288
Epoch 8/10
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9920 - loss: 0.2443 - val_accuracy: 0.9459 - v

**est the remaining 20% of the data, report test and training accuracy (5 Marks)**

In [15]:
train_acc = history.history['accuracy'][-1]
test_acc = history.history['val_accuracy'][-1]

print(f"Training Accuracy: {train_acc:.4f}")
print(f"Testing Accuracy: {test_acc:.4f}")

Training Accuracy: 0.9966
Testing Accuracy: 0.9595


In [16]:
from sklearn.metrics import classification_report
import numpy as np

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print(classification_report(y_true_classes, y_pred_classes, target_names=label_encoder.classes_))

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
              precision    recall  f1-score   support

   athletics       1.00      1.00      1.00        20
     cricket       1.00      0.84      0.91        25
    football       0.96      0.98      0.97        53
       rugby       0.88      1.00      0.94        30
      tennis       1.00      0.95      0.97        20

    accuracy                           0.96       148
   macro avg       0.97      0.95      0.96       148
weighted avg       0.96      0.96      0.96       148



Inside your note book, also answer following questions, (5 Marks)
1. Does accuracy vary from class to class label?
2. If so, write the reasons



1.   yes, some classes achieve higher accuracy like footbal than others
2.  Imbalanced data: Some classes may have more training examples.
3. Vocabulary overlap: Words in football and cricket may be more distinct, making them easier to classify.
4. Ambiguity: News stories about athletics and rugby may use more general words, making classification harder.



